In [27]:
import os
import pandas as pd

In [28]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_logician/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_logician/')

In [29]:
enz_sub = pd.read_csv('data/external/cantley/johnson_supp3_table.csv', index_col = 0)

In [30]:
psts = []
for cline in ['HL60', 'MCF7', 'NTERA2']:
    obs = pd.read_csv(f'data/processed/facts/observations_mp_{cline}.csv', sep=',')
    psts = list(set((psts + obs['pst'].tolist())))

In [31]:
# filter enz_sub to only include rows where Gene is not NaN and reset index
enz_sub = enz_sub[enz_sub['Gene'].notna()]
# remove rows where Gene contains ';'
enz_sub = enz_sub[~enz_sub['Gene'].str.contains(';')]
# merge Gene and phosphosite columns into a new column called pst
enz_sub['pst'] = enz_sub['Gene'] + '(' + enz_sub['Phosphosite'] + ')' 
# remove rows where pst not in psts
enz_sub = enz_sub[enz_sub['pst'].isin(psts)].reset_index(drop=True)
# rename Gene column to tprot
enz_sub = enz_sub.rename(columns={'Gene': 'tprot'})
# find column names in enz_sub ending in _percentile using regex and save as list
percentile_cols = enz_sub.filter(regex='_percentile$', axis=1).columns.tolist()
# remove median_percentile column from list
percentile_cols.remove('median_percentile')
# filter enz_sub to only include pst, tprot and percentile columns
enz_sub = enz_sub[['pst', 'tprot'] + percentile_cols]
# stack dataframe with pst and tprot as id_vars and percentile columns as value_vars
enz_sub = enz_sub.melt(id_vars=['pst', 'tprot'], var_name='kpa', value_name='percentile')
# remove _percentile from kpa column
enz_sub['kpa'] = enz_sub['kpa'].str.replace('_percentile', '')

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [32]:
enz_sub

,pst,tprot,kpa,percentile
0,E2F8(S358),E2F8,AAK1,22.61
1,E2F8(S417),E2F8,AAK1,39.60
2,E2F8(S71),E2F8,AAK1,67.93
3,UBA6(S737),UBA6,AAK1,50.88
4,ESYT2(S688),ESYT2,AAK1,33.01
...,...,...,...,...
3511765,MORC2(S777),MORC2,ZAK,31.48
3511766,MORC2(S779),MORC2,ZAK,31.92
3511767,IVNS1ABP(S277),IVNS1ABP,ZAK,49.81
3511768,IVNS1ABP(S322),IVNS1ABP,ZAK,51.73


In [33]:
enz_sub = enz_sub.drop_duplicates()
enz_sub = enz_sub.sort_values('percentile').drop_duplicates(['kpa', 'pst'], keep='last')
enz_sub

,pst,tprot,kpa,percentile
3449819,PCNP(S119),PCNP,WNK4,0.0
2344001,RANBP2(S2900),RANBP2,P70S6K,0.0
2230079,SEPTIN7(T228),SEPTIN7,NIM1,0.0
303221,NCL(S28),NCL,BUB1,0.0
1462385,MCM3(T674),MCM3,IRE1,0.0
...,...,...,...,...
160615,TMEM51(S115),TMEM51,ATM,100.0
2043960,ATR(S435),ATR,MYLK4,100.0
1237235,NAP1L4(S125),NAP1L4,GRK5,100.0
1259309,PCNP(S119),PCNP,GRK7,100.0


In [21]:
# export df to csv
enz_sub.to_csv('data/processed/facts/enz_sub_cantley.csv', index=False)

In [36]:
for p in [60, 70, 80, 90, 95, 99, 99.5]:
    # filter enz_sub to only include rows where percentile is greater than p%
    enz_sub_filt = enz_sub[enz_sub['percentile'] > p]

    # write Prolog file
    with open(f'models/facts/cantley/enz_sub_cantley_p{p}.pl', 'w') as file:
        for index, row in enz_sub_filt.iterrows():
            var1 = "enz_sub('{}', '{}', '{}').".format(row['kpa'], row['pst'], row['tprot'])
            file.write(var1 + '\n')

OSError: [Errno 28] No space left on device